This project will determine the best kind of movies for Microsoft to make.
First we will import the necessary packages, then we will load the web page to scrape.

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [3]:
html_page = requests.get('https://www.boxofficemojo.com/chart/top_lifetime_gross/?area=XWW')
soup = BeautifulSoup(html_page.content, 'html.parser') 

In [4]:
soup.prettify

<bound method Tag.prettify of <!DOCTYPE doctype html>
<html class="a-no-js" data-19ax5a9jf="dingo"><head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta charset="utf-8"/>
<title dir="ltr">Top Lifetime Grosses - Box Office Mojo</title><meta content="Top Lifetime Grosses" name="title"/>
<meta content="Box Office Mojo" property="og:site_name"/>
<meta content="telephone=no" name="format-detection"/>
<link href="https://m.media-amazon.com/images/G/01/boxofficemojo/v2/favicon._CB448965889_.ico" rel="icon" type="image/x-icon"/>
<link href="https://images-na.ssl-images-amazon.com/images/I/11EIQ5IGqaL._RC|012LjolmrML.css,41t4Ok2rjiL.css,51IB+wfP8qL.css,01evdoiemkL.css,01oDR3IULNL.css,01Vctty9pOL.css,31zpKVx8wkL.css,01XPHJk60-L.css,21qPwhPKAAL.css,01Jb-VvL4uL.css,21LK7jaicML.css,11L58Qpo0GL.css,21kyTi1FabL.css,01ruG+gDPFL.css,01egbgkonWL.css,21GwE3cR-yL.css,119kww4uFyL.css,11nWWh1kQdL.cs

The table consists of four types of data: rank, title, lifetime gross, and year. Using the inspect element feature 
in the browser, it is evident that the rank is in mojo-field-type-rank; title in mojo-field-type-title, lifetime 
gross in mojo-field-type-rank, and release year in mojo-field-type-year

In [4]:
rank = soup.find('td', class_="mojo-field-type-rank")
rank 

<td class="a-text-right mojo-header-column mojo-truncate mojo-field-type-rank">1</td>

In [169]:
rank_list = [r.text for r in soup.findAll('td', class_="mojo-field-type-rank")] 
print(len(rank_list), rank_list[:5])

200 ['1', '2', '3', '4', '5']


In [6]:
title = rank.nextSibling
title

<td class="a-text-left mojo-field-type-title"><a class="a-link-normal" href="/title/tt4154796/?ref_=bo_cso_table_1">Avengers: Endgame</a></td>

In [170]:
title_list = [t.text for t in soup.findAll('td', class_="mojo-field-type-title")] 
print(len(title_list), title_list[:5])

200 ['Avengers: Endgame', 'Avatar', 'Titanic', 'Star Wars: Episode VII - The Force Awakens', 'Avengers: Infinity War']


In [8]:
money = title.nextSibling
money

<td class="a-text-right mojo-field-type-money">$2,797,800,564</td>

In [171]:
money_list = [m.text for m in soup.findAll('td', class_="mojo-field-type-money")] 
print(len(money_list), money_list[:5])

200 ['$2,797,800,564', '$2,790,439,092', '$2,471,754,307', '$2,068,454,310', '$2,048,359,754']


In [10]:
year = money.nextSibling
year

<td class="a-text-left mojo-field-type-year"><a class="a-link-normal" href="/year/world/2019/?ref_=bo_cso_table_1">2019</a></td>

In [172]:
year_list = [y.text for y in soup.findAll('td', class_="mojo-field-type-year")] 
print(len(year_list), year_list[:5])

200 ['2019', '2009', '1997', '2015', '2018']


In [173]:
# The links to the movie pages didn't have an attribute that was unique (it was the same for every link in the page). 
# To get just the link, first I got a container with the url we need for each movie
link_list = [t for t in soup.findAll('td', class_="mojo-field-type-title")] 
print(len(link_list), link_list[0])

200 <td class="a-text-left mojo-field-type-title"><a class="a-link-normal" href="/title/tt4154796/?ref_=bo_cso_table_1">Avengers: Endgame</a></td>


In [174]:
# Then I extracted just the href from each 
new_link_list = []
for t in link_list:
    link = [x.get('href') for x in t.findAll('a', class_="a-link-normal")]
    link_str = link[0]
    new_link_list.append(link_str)
print(len(new_link_list), new_link_list[0])

200 /title/tt4154796/?ref_=bo_cso_table_1


In [175]:
new_link_list[:5]

['/title/tt4154796/?ref_=bo_cso_table_1',
 '/title/tt0499549/?ref_=bo_cso_table_2',
 '/title/tt0120338/?ref_=bo_cso_table_3',
 '/title/tt2488496/?ref_=bo_cso_table_4',
 '/title/tt4154756/?ref_=bo_cso_table_5']

In [206]:
df = pd.DataFrame([rank_list, title_list, new_link_list, money_list, year_list]).transpose()
df.columns = ['Lifetime_Rank', 'Movie_Title', 'Movie_Page', 'Lifetime_Gross', 'Year_Released']
df

,Lifetime_Rank,Movie_Title,Movie_Page,Lifetime_Gross,Year_Released
0,1,Avengers: Endgame,/title/tt4154796/?ref_=bo_cso_table_1,"$2,797,800,564",2019
1,2,Avatar,/title/tt0499549/?ref_=bo_cso_table_2,"$2,790,439,092",2009
2,3,Titanic,/title/tt0120338/?ref_=bo_cso_table_3,"$2,471,754,307",1997
3,4,Star Wars: Episode VII - The Force Awakens,/title/tt2488496/?ref_=bo_cso_table_4,"$2,068,454,310",2015
4,5,Avengers: Infinity War,/title/tt4154756/?ref_=bo_cso_table_5,"$2,048,359,754",2018
...,...,...,...,...,...
195,196,The Simpsons Movie,/title/tt0462538/?ref_=bo_cso_table_196,"$536,414,293",2007
196,197,The Revenant,/title/tt1663202/?ref_=bo_cso_table_197,"$532,950,503",2015
197,198,The Meg,/title/tt4779682/?ref_=bo_cso_table_198,"$530,259,473",2018
198,199,Ralph Breaks the Internet,/title/tt5848272/?ref_=bo_cso_table_199,"$529,323,962",2018


In [180]:
# Get top grossing films, 2019, 2020
html_page2 = requests.get('https://www.boxofficemojo.com/year/2019/?ref_=bo_yl_table_3')
soup2 = BeautifulSoup(html_page2.content, 'html.parser') 
rank_2019 = soup2.find('td', class_="mojo-field-type-rank")
title_2019 = soup2.find('td', class_="mojo-field-type-release")
money_2019 = soup2.find('td', class_="mojo-field-type-money")
print(rank_2019, title_2019, money_2019)

<td class="a-text-right mojo-header-column mojo-truncate mojo-field-type-rank mojo-sort-column">1</td> <td class="a-text-left mojo-field-type-release mojo-cell-wide"><a class="a-link-normal" href="/release/rl3059975681/?ref_=bo_yld_table_1">Avengers: Endgame</a></td> <td class="a-text-right mojo-field-type-money hidden">-</td>


In [181]:
rank_2019_list = [r.text for r in soup2.findAll('td', class_="mojo-field-type-rank")] 
print(len(rank_2019_list), rank_2019_list[:5])
title_2019_list = [t.text for t in soup2.findAll('td', class_="mojo-field-type-release")] 
print(len(title_2019_list), title_2019_list[:5])
money_2019_list = [m.text for m in soup2.findAll('td', class_="mojo-field-type-money")] 
print(len(money_2019_list), money_2019_list[:5])

911 ['1', '2', '3', '4', '5']
911 ['Avengers: Endgame', 'The Lion King', 'Toy Story 4', 'Frozen II', 'Captain Marvel']
2733 ['-', '$858,373,000', '$858,373,000', '-', '$543,638,043']


In [182]:
revised_money_2019_list = []
for i in range(len(money_2019_list)):
    if i%3 == 1:
       revised_money_2019_list.append(money_2019_list[i])
print(len(revised_money_2019_list), revised_money_2019_list[:5])

911 ['$858,373,000', '$543,638,043', '$434,038,008', '$430,144,682', '$426,829,839']


In [193]:
# as in the first dataframe, I extracted the link to the page for each movie
link_list_2019 = [t for t in soup2.findAll('td', class_="mojo-field-type-release")] 
new_link_list_2019 = []
for t in link_list_2019:
    link = [x.get('href') for x in t.findAll('a', class_="a-link-normal")]
    link_str = link[0]
    new_link_list_2019.append(link_str)
print(len(new_link_list_2019), new_link_list_2019[0])

911 /release/rl3059975681/?ref_=bo_yld_table_1


In [200]:
df_2019 = pd.DataFrame([rank_2019_list, title_2019_list, new_link_list_2019, revised_money_2019_list]).transpose()
df_2019.columns = ['2019_Rank', 'Movie_Title', 'Movie_Page', '2019_Gross']
df_2019

,2019_Rank,Movie_Title,Movie_Page,2019_Gross
0,1,Avengers: Endgame,/release/rl3059975681/?ref_=bo_yld_table_1,"$858,373,000"
1,2,The Lion King,/release/rl3321923073/?ref_=bo_yld_table_2,"$543,638,043"
2,3,Toy Story 4,/release/rl3798500865/?ref_=bo_yld_table_3,"$434,038,008"
3,4,Frozen II,/release/rl2424210945/?ref_=bo_yld_table_4,"$430,144,682"
4,5,Captain Marvel,/release/rl3009644033/?ref_=bo_yld_table_5,"$426,829,839"
...,...,...,...,...
906,907,Henchmen,/release/rl1241941505/?ref_=bo_yld_table_907,$297
907,908,1945,/release/rl3506603521/?ref_=bo_yld_table_908,$236
908,909,Tall Tales from the Magical Garden of Antoon K...,/release/rl17401345/?ref_=bo_yld_table_909,$220
909,910,Over the Limit,/release/rl268928513/?ref_=bo_yld_table_910,$117


In [201]:
# get the page
html_page3 = requests.get('https://www.boxofficemojo.com/year/2020/?grossesOption=calendarGrosses')
soup3 = BeautifulSoup(html_page3.content, 'html.parser') 

# retrieve the data for each column
rank_2020_list = [r.text for r in soup3.findAll('td', class_="mojo-field-type-rank")] 
title_2020_list = [t.text for t in soup3.findAll('td', class_="mojo-field-type-release")] 
money_2020_list = [m.text for m in soup3.findAll('td', class_="mojo-field-type-money")] 

# select only the money column of that year's gross
revised_money_2020_list = []
for i in range(len(money_2020_list)):
    if i%3 == 1:
       revised_money_2020_list.append(money_2020_list[i])
    
# as in the first dataframe, I extracted the link to the page for each movie
link_list_2020 = [t for t in soup3.findAll('td', class_="mojo-field-type-release")] 
new_link_list_2020 = []
for t in link_list_2020:
    link = [x.get('href') for x in t.findAll('a', class_="a-link-normal")]
    link_str = link[0]
    new_link_list_2020.append(link_str)
print(len(new_link_list_2020), new_link_list_2020[0])

# create the dataframe
df_2020 = pd.DataFrame([rank_2020_list, title_2020_list, new_link_list_2020, revised_money_2020_list]).transpose()
df_2020.columns = ['2020_Rank', 'Movie_Title', 'Movie_Page', '2020_Gross']
df_2020

454 /release/rl1182631425/?ref_=bo_yld_table_1


,2020_Rank,Movie_Title,Movie_Page,2020_Gross
0,1,Bad Boys for Life,/release/rl1182631425/?ref_=bo_yld_table_1,"$204,417,855"
1,2,1917,/release/rl2969994753/?ref_=bo_yld_table_2,"$157,901,466"
2,3,Sonic the Hedgehog,/release/rl4244997633/?ref_=bo_yld_table_3,"$146,066,470"
3,4,Jumanji: The Next Level,/release/rl755467777/?ref_=bo_yld_table_4,"$124,736,710"
4,5,Star Wars: Episode IX - The Rise of Skywalker,/release/rl3305145857/?ref_=bo_yld_table_5,"$124,496,308"
...,...,...,...,...
449,450,Asako I & II,/release/rl1359513089/?ref_=bo_yld_table_450,$231
450,451,Chained for Life,/release/rl1292600833/?ref_=bo_yld_table_451,$115
451,452,Shooting the Mafia,/release/rl2366342657/?ref_=bo_yld_table_452,$88
452,453,Benjamin the Elephant (2020),/release/rl1180795649/?ref_=bo_yld_table_453,$49


In [202]:
# limited my df to the top 200 movies
df_2020 = df_2020.drop(df_2020.index[200:])
df_2020

,2020_Rank,Movie_Title,Movie_Page,2020_Gross
0,1,Bad Boys for Life,/release/rl1182631425/?ref_=bo_yld_table_1,"$204,417,855"
1,2,1917,/release/rl2969994753/?ref_=bo_yld_table_2,"$157,901,466"
2,3,Sonic the Hedgehog,/release/rl4244997633/?ref_=bo_yld_table_3,"$146,066,470"
3,4,Jumanji: The Next Level,/release/rl755467777/?ref_=bo_yld_table_4,"$124,736,710"
4,5,Star Wars: Episode IX - The Rise of Skywalker,/release/rl3305145857/?ref_=bo_yld_table_5,"$124,496,308"
...,...,...,...,...
195,196,Extra Ordinary,/release/rl2562359809/?ref_=bo_yld_table_196,"$164,346"
196,197,Palm Springs,/release/rl3175187201/?ref_=bo_yld_table_197,"$164,000"
197,198,A Call to Spy,/release/rl3796140801/?ref_=bo_yld_table_198,"$158,864"
198,199,Love Story50th Anniversary,/release/rl934707713/?ref_=bo_yld_table_199,"$153,504"


In [203]:
# limited my df to the top 200 movies

df_2019 = df_2019.drop(df_2019.index[200:])
df_2019

,2019_Rank,Movie_Title,Movie_Page,2019_Gross
0,1,Avengers: Endgame,/release/rl3059975681/?ref_=bo_yld_table_1,"$858,373,000"
1,2,The Lion King,/release/rl3321923073/?ref_=bo_yld_table_2,"$543,638,043"
2,3,Toy Story 4,/release/rl3798500865/?ref_=bo_yld_table_3,"$434,038,008"
3,4,Frozen II,/release/rl2424210945/?ref_=bo_yld_table_4,"$430,144,682"
4,5,Captain Marvel,/release/rl3009644033/?ref_=bo_yld_table_5,"$426,829,839"
...,...,...,...,...
195,196,Welcome to Marwen,/release/rl2264958465/?ref_=bo_yld_table_196,"$2,405,625"
196,197,"My People, My Country",/release/rl2433451521/?ref_=bo_yld_table_197,"$2,356,683"
197,198,Promare,/release/rl3104540161/?ref_=bo_yld_table_198,"$2,287,950"
198,199,Super 30,/release/rl4211639809/?ref_=bo_yld_table_199,"$2,269,878"


200

In [284]:
# get budget, MPAA, genre

import re

html_page4 = requests.get('https://www.boxofficemojo.com/title/tt4154796/?ref_=bo_cso_table_1')
soup4 = BeautifulSoup(html_page4.content, 'html.parser')
# the domestic gross if the first span tag
gross = soup4.find('span', class_="money")
gross = str(gross)
gross = gross[20:-7]
gross

# the budget is not the first span tag with the money class, so it took extra steps
budget = soup4.find_all('span', text=re.compile('Budget')) 
budget_parent = [budget.parent for budget in budget]
budget_parent_string = budget_parent[0]
budget_parent_string
budget_money = [m.text for m in budget_parent_string.findAll('span', class_="money")]
budget_money_string = budget_money[0]
budget_money_string

# likewise for MPAA rating
mpaa = soup4.find_all('span', text=re.compile('MPAA')) 
mpaa_parent = [mpaa.parent for mpaa in mpaa]
mpaa_parent_string = mpaa_parent[0]
mpaa_rating = [m.text for m in mpaa_parent_string.findAll('span')]
mpaa_rating_string = mpaa_rating[1]
mpaa_rating_string

#likewise for genres
genres = soup4.find_all('span', text=re.compile('Genres')) 
genres_parent = [genres.parent for genres in genres]
genres_parent_string = genres_parent[0]
genres_categories = [m.text for m in genres_parent_string.findAll('span')]
genres_categories_string = genres_categories[1]
genres_categories_string



'Action\n    \n        Adventure\n    \n        Drama\n    \n        Sci-Fi'

In [285]:
#make genre categories a list
genres_list = list(genres_categories_string.split("\n    \n        "))
genres_list

['Action', 'Adventure', 'Drama', 'Sci-Fi']

'/title/tt4154796/?ref_=bo_cso_table_1'

In [563]:
# movie_url_list = []
# gross_master_list = []
# budget_master_list = []
# mpaa_master_list = []
# genres_master_list = []

j=500
    
for m in df_new2.Movie_Title:
    movie_url = 'https://www.boxofficemojo.com' + df_new2.Movie_Page_x[j]
#     movie_url_list.append(movie_url)
    html_page5 = requests.get(movie_url)
    soup5 = BeautifulSoup(html_page5.content, 'html.parser')
    
# the domestic gross from the first span tag
    gross = soup5.find('span', class_="money")
    gross = str(gross)
    gross = gross[20:-7]

# the budget is not the first span tag with the money class, so it took extra steps
    budget = soup5.find_all('span', text=re.compile('Budget')) 
    budget_parent = [budget.parent for budget in budget]
    if not budget_parent:
        budget_money = [np.nan]
    else:
        budget_parent_string = budget_parent[0]
        budget_money = [m.text for m in budget_parent_string.findAll('span', class_="money")]
        budget_money_string = budget_money[0]

# likewise for MPAA rating
    mpaa = soup5.find_all('span', text=re.compile('MPAA')) 
    mpaa_parent = [mpaa.parent for mpaa in mpaa]
    if not mpaa_parent:
        mpaa_rating = [np.nan, np.nan]
    else:
        mpaa_parent_string = mpaa_parent[0]
        mpaa_rating = [m.text for m in mpaa_parent_string.findAll('span')]
        mpaa_rating_string = mpaa_rating[1]

#likewise for genres
    genres = soup5.find_all('span', text=re.compile('Genres')) 
    genres_parent = [genres.parent for genres in genres]
    if not genres_parent:
        genres_categories = [np.nan, np.nan]
    else:
        genres_parent_string = genres_parent[0]
        genres_categories = [m.text for m in genres_parent_string.findAll('span')]
        genres_categories_string = genres_categories[1]

    gross_master_list.append(gross)
    budget_master_list.append(budget_money_string)
    mpaa_master_list.append(mpaa_rating_string)
    genres_master_list.append(genres_categories_string)
    
    j += 1
    if j > 550:
        break
 
# j, gross_master_list, budget_master_list, mpaa_master_list, genres_master_list

In [578]:
#make genre categories a list
movie_genres_list = []
genres_master_list_split = []
for x in genres_master_list_to_split:
    movie_genres_list = list(x.split("\n    \n        "))
    genres_master_list_split.append(movie_genres_list)
genres_master_list_split[:5]

['Action\n    \n        Adventure\n    \n        Drama\n    \n        Sci-Fi',
 'Action\n    \n        Adventure\n    \n        Fantasy\n    \n        Sci-Fi',
 'Drama\n    \n        Romance',
 'Action\n    \n        Adventure\n    \n        Sci-Fi',
 'Action\n    \n        Adventure\n    \n        Sci-Fi']

In [588]:
len(gross_master_list), len(budget_master_list), len(mpaa_master_list), len(genres_master_list_split)

(551, 551, 551, 551)

In [593]:
# add lists of scraped data to the master dataframe
df_new3 = df_new2
df_new3['Domestic_Gross']= gross_master_list
df_new3['Budget'] = budget_master_list
df_new3['MPAA'] = mpaa_master_list
df_new3['Genres'] = genres_master_list_split
df_new3.head()

,Lifetime_Rank,Movie_Title,Movie_Page_x,Lifetime_Gross,Year_Released,2019_Rank,Movie_Page_y,2019_Gross,2020_Rank,Movie_Page,2020_Gross,Domestic_Gross,Budget,MPAA,Genres
0,1,Avengers: Endgame,/title/tt4154796/?ref_=bo_cso_table_1,"$2,797,800,564",2019,1,/release/rl3059975681/?ref_=bo_yld_table_1,"$858,373,000",NaN,NaN,NaN,"$858,373,000","$356,000,000",PG-13,"[Action, Adventure, Drama, Sci-Fi]"
1,2,Avatar,/title/tt0499549/?ref_=bo_cso_table_2,"$2,790,439,092",2009,NaN,NaN,NaN,NaN,NaN,NaN,"$760,507,625","$237,000,000",PG-13,"[Action, Adventure, Fantasy, Sci-Fi]"
2,3,Titanic,/title/tt0120338/?ref_=bo_cso_table_3,"$2,471,754,307",1997,NaN,NaN,NaN,NaN,NaN,NaN,"$659,363,944","$200,000,000",PG-13,"[Drama, Romance]"
3,4,Star Wars: Episode VII - The Force Awakens,/title/tt2488496/?ref_=bo_cso_table_4,"$2,068,454,310",2015,NaN,NaN,NaN,NaN,NaN,NaN,"$936,662,225","$245,000,000",PG-13,"[Action, Adventure, Sci-Fi]"
4,5,Avengers: Infinity War,/title/tt4154756/?ref_=bo_cso_table_5,"$2,048,359,754",2018,NaN,NaN,NaN,NaN,NaN,NaN,"$678,815,482","$245,000,000",PG-13,"[Action, Adventure, Sci-Fi]"


In [635]:
df_new3.head()

,Lifetime_Rank,Movie_Title,Movie_Page_x,Lifetime_Gross,Year_Released,2019_Rank,Movie_Page_y,2019_Gross,2020_Rank,Movie_Page,2020_Gross,Domestic_Gross,Budget,MPAA,Genres
0,1,Avengers: Endgame,/title/tt4154796/?ref_=bo_cso_table_1,"$2,797,800,564",2019,1,/release/rl3059975681/?ref_=bo_yld_table_1,"$858,373,000",NaN,NaN,NaN,"$858,373,000","$356,000,000",PG-13,"[Action, Adventure, Drama, Sci-Fi]"
1,2,Avatar,/title/tt0499549/?ref_=bo_cso_table_2,"$2,790,439,092",2009,NaN,NaN,NaN,NaN,NaN,NaN,"$760,507,625","$237,000,000",PG-13,"[Action, Adventure, Fantasy, Sci-Fi]"
2,3,Titanic,/title/tt0120338/?ref_=bo_cso_table_3,"$2,471,754,307",1997,NaN,NaN,NaN,NaN,NaN,NaN,"$659,363,944","$200,000,000",PG-13,"[Drama, Romance]"
3,4,Star Wars: Episode VII - The Force Awakens,/title/tt2488496/?ref_=bo_cso_table_4,"$2,068,454,310",2015,NaN,NaN,NaN,NaN,NaN,NaN,"$936,662,225","$245,000,000",PG-13,"[Action, Adventure, Sci-Fi]"
4,5,Avengers: Infinity War,/title/tt4154756/?ref_=bo_cso_table_5,"$2,048,359,754",2018,NaN,NaN,NaN,NaN,NaN,NaN,"$678,815,482","$245,000,000",PG-13,"[Action, Adventure, Sci-Fi]"


In [634]:
i=0
j=0
for x in df_new3:
    if not df_new3['2020_Rank'][i] == None:
        print(df_new3.Movie_Title[i])
    i +=1


j

Avengers: Endgame
Avatar
Titanic
Star Wars: Episode VII - The Force Awakens
Avengers: Infinity War
Jurassic World
The Lion King
The Lion King
The Avengers
Furious 7
Frozen II
Avengers: Age of Ultron
Black Panther
Harry Potter and the Deathly Hallows: Part 2
Star Wars: Episode VIII - The Last Jedi


0

In [636]:
if not df_new3['2020_Rank'][0] == None:
        print(df_new3.Movie_Title[0])

Avengers: Endgame


In [637]:
pd.isnull(df_new3['2020_Rank'].iloc[0])

True

In [569]:
df_new2.tail()

,Lifetime_Rank,Movie_Title,Movie_Page_x,Lifetime_Gross,Year_Released,2019_Rank,Movie_Page_y,2019_Gross,2020_Rank,Movie_Page,2020_Gross
546,NaN,Extra Ordinary,/release/rl2562359809/?ref_=bo_yld_table_196,NaN,NaN,NaN,NaN,NaN,196,/release/rl2562359809/?ref_=bo_yld_table_196,"$164,346"
547,NaN,Palm Springs,/release/rl3175187201/?ref_=bo_yld_table_197,NaN,NaN,NaN,NaN,NaN,197,/release/rl3175187201/?ref_=bo_yld_table_197,"$164,000"
548,NaN,A Call to Spy,/release/rl3796140801/?ref_=bo_yld_table_198,NaN,NaN,NaN,NaN,NaN,198,/release/rl3796140801/?ref_=bo_yld_table_198,"$158,864"
549,NaN,Love Story50th Anniversary,/release/rl934707713/?ref_=bo_yld_table_199,NaN,NaN,NaN,NaN,NaN,199,/release/rl934707713/?ref_=bo_yld_table_199,"$153,504"
550,NaN,The Dark and the Wicked,/release/rl4015883009/?ref_=bo_yld_table_200,NaN,NaN,NaN,NaN,NaN,200,/release/rl4015883009/?ref_=bo_yld_table_200,"$149,011"


In [570]:
df_scraped_data = pd.DataFrame(list(zip(gross_master_list, budget_master_list, mpaa_master_list, genres_master_list)), 
               columns =['Domestic_Gross', 'Budget', 'MPAA', 'Genres']) 
df_scraped_data.head()

,Domestic_Gross,Budget,MPAA,Genres
0,"$858,373,000","$356,000,000",PG-13,Action\n \n Adventure\n \n ...
1,"$760,507,625","$237,000,000",PG-13,Action\n \n Adventure\n \n ...
2,"$659,363,944","$200,000,000",PG-13,Drama\n \n Romance
3,"$936,662,225","$245,000,000",PG-13,Action\n \n Adventure\n \n ...
4,"$678,815,482","$245,000,000",PG-13,Action\n \n Adventure\n \n ...


In [573]:
conn = sqlite3.connect('scraped_data.db') 
c = conn.cursor()

# c.execute('CREATE TABLE scraped_data (Domestic_Gross, Budget, MPAA, Genres)')
# conn.commit()

df_scraped_data.to_sql('scraped_data', conn, if_exists='replace', index = False)


In [576]:
cnx = sqlite3.connect('scraped_data.db')

# df_imported = pd.read_sql_query("SELECT * FROM scraped_data", cnx)
df_imported.head()

,Domestic_Gross,Budget,MPAA,Genres
0,"$858,373,000","$356,000,000",PG-13,Action\n \n Adventure\n \n ...
1,"$760,507,625","$237,000,000",PG-13,Action\n \n Adventure\n \n ...
2,"$659,363,944","$200,000,000",PG-13,Drama\n \n Romance
3,"$936,662,225","$245,000,000",PG-13,Action\n \n Adventure\n \n ...
4,"$678,815,482","$245,000,000",PG-13,Action\n \n Adventure\n \n ...


In [691]:
df_new3.groupby('MPAA').sum()

,Movie_Title,Movie_Page_x,Year_Released,Domestic_Gross,Budget,Genres
MPAA,,,,,,
G,The Lion KingTransformers: Age of ExtinctionTo...,/title/tt6105098/?ref_=bo_cso_table_7/title/tt...,2019201420192010201320072001201820112019201920...,"$422,783,777$422,783,777$434,038,008$415,004,8...","$45,000,000$45,000,000$200,000,000$200,000,000...","[Adventure, Animation, Drama, Family, Musical,..."
Not Rated,BharatKalank,/release/rl654870017/?ref_=bo_yld_table_185/re...,20192019,"$2,971,549$2,729,336","$50,000,000$50,000,000","[Action, Comedy, Drama, Drama, Romance]"
PG,Jurassic WorldFurious 7Jurassic World: Fallen ...,/title/tt0369610/?ref_=bo_cso_table_6/title/tt...,2015201520182013201720132019201720161999201020...,"$543,638,043$477,373,578$400,953,009$504,481,1...","$260,000,000$150,000,000$150,000,000$160,000,0...","[Adventure, Animation, Drama, Family, Musical,..."
PG-13,Avengers: EndgameAvatarTitanicStar Wars: Episo...,/title/tt4154796/?ref_=bo_cso_table_1/title/tt...,2019200919972015201820192012201920152018201120...,"$858,373,000$760,507,625$659,363,944$936,662,2...","$356,000,000$237,000,000$200,000,000$245,000,0...","[Action, Adventure, Drama, Sci-Fi, Action, Adv..."
R,JokerDeadpool 2DeadpoolThe Matrix ReloadedItIt...,/title/tt7286456/?ref_=bo_cso_table_33/title/t...,2019201820162003201720192017200420112015201620...,"$335,451,311$324,591,735$363,070,709$281,576,4...","$55,000,000$110,000,000$58,000,000$150,000,000...","[Crime, Drama, Thriller, Action, Adventure, Co..."


In [692]:
df_new3.Lifetime_Gross[0]

'$2,797,800,564'

[('1',
  'Avengers: Endgame',
  '/title/tt4154796/?ref_=bo_cso_table_1',
  '$2,797,800,564',
  '2019'),
 ('2',
  'Avatar',
  '/title/tt0499549/?ref_=bo_cso_table_2',
  '$2,790,439,092',
  '2009'),
 ('3',
  'Titanic',
  '/title/tt0120338/?ref_=bo_cso_table_3',
  '$2,471,754,307',
  '1997'),
 ('4',
  'Star Wars: Episode VII - The Force Awakens',
  '/title/tt2488496/?ref_=bo_cso_table_4',
  '$2,068,454,310',
  '2015'),
 ('5',
  'Avengers: Infinity War',
  '/title/tt4154756/?ref_=bo_cso_table_5',
  '$2,048,359,754',
  '2018'),
 ('6',
  'Jurassic World',
  '/title/tt0369610/?ref_=bo_cso_table_6',
  '$1,670,471,444',
  '2015'),
 ('7',
  'The Lion King',
  '/title/tt6105098/?ref_=bo_cso_table_7',
  '$1,657,870,986',
  '2019'),
 ('8',
  'The Avengers',
  '/title/tt0848228/?ref_=bo_cso_table_8',
  '$1,518,815,515',
  '2012'),
 ('9',
  'Furious 7',
  '/title/tt2820852/?ref_=bo_cso_table_9',
  '$1,515,255,622',
  '2015'),
 ('10',
  'Frozen II',
  '/title/tt4520988/?ref_=bo_cso_table_10',
  '$1,45

[('1',
  'Avengers: Endgame',
  '/title/tt4154796/?ref_=bo_cso_table_1',
  '$2,797,800,564',
  '2019',
  'Avengers: Endgame',
  '/title/tt4154796/?ref_=bo_cso_table_1'),
 ('2',
  'Avatar',
  '/title/tt0499549/?ref_=bo_cso_table_2',
  '$2,790,439,092',
  '2009',
  'Avatar',
  '/title/tt0499549/?ref_=bo_cso_table_2'),
 ('3',
  'Titanic',
  '/title/tt0120338/?ref_=bo_cso_table_3',
  '$2,471,754,307',
  '1997',
  'Titanic',
  '/title/tt0120338/?ref_=bo_cso_table_3'),
 ('4',
  'Star Wars: Episode VII - The Force Awakens',
  '/title/tt2488496/?ref_=bo_cso_table_4',
  '$2,068,454,310',
  '2015',
  'Star Wars: Episode VII - The Force Awakens',
  '/title/tt2488496/?ref_=bo_cso_table_4'),
 ('5',
  'Avengers: Infinity War',
  '/title/tt4154756/?ref_=bo_cso_table_5',
  '$2,048,359,754',
  '2018',
  'Avengers: Infinity War',
  '/title/tt4154756/?ref_=bo_cso_table_5'),
 ('6',
  'Jurassic World',
  '/title/tt0369610/?ref_=bo_cso_table_6',
  '$1,670,471,444',
  '2015',
  'Jurassic World',
  '/title/tt

q = "SELECT * FROM db_master ;"

df_combined = pd.read_sql_query(q,conn)
df_combined.head()

200

In [348]:
  
# joining the DataFrames 
df_new = pd.merge(df, df_2019, on = "Movie_Title", how = "outer") 
len(df_new)

382

In [349]:
df_new.head()

,Lifetime_Rank,Movie_Title,Movie_Page_x,Lifetime_Gross,Year_Released,2019_Rank,Movie_Page_y,2019_Gross
0,1,Avengers: Endgame,/title/tt4154796/?ref_=bo_cso_table_1,"$2,797,800,564",2019,1,/release/rl3059975681/?ref_=bo_yld_table_1,"$858,373,000"
1,2,Avatar,/title/tt0499549/?ref_=bo_cso_table_2,"$2,790,439,092",2009,NaN,NaN,NaN
2,3,Titanic,/title/tt0120338/?ref_=bo_cso_table_3,"$2,471,754,307",1997,NaN,NaN,NaN
3,4,Star Wars: Episode VII - The Force Awakens,/title/tt2488496/?ref_=bo_cso_table_4,"$2,068,454,310",2015,NaN,NaN,NaN
4,5,Avengers: Infinity War,/title/tt4154756/?ref_=bo_cso_table_5,"$2,048,359,754",2018,NaN,NaN,NaN


In [350]:
df_new2 = pd.merge(df_new, df_2020, on = "Movie_Title", how = "outer") 
len(df_new2)

551

In [351]:
df_new2.head()

,Lifetime_Rank,Movie_Title,Movie_Page_x,Lifetime_Gross,Year_Released,2019_Rank,Movie_Page_y,2019_Gross,2020_Rank,Movie_Page,2020_Gross
0,1,Avengers: Endgame,/title/tt4154796/?ref_=bo_cso_table_1,"$2,797,800,564",2019,1,/release/rl3059975681/?ref_=bo_yld_table_1,"$858,373,000",NaN,NaN,NaN
1,2,Avatar,/title/tt0499549/?ref_=bo_cso_table_2,"$2,790,439,092",2009,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Titanic,/title/tt0120338/?ref_=bo_cso_table_3,"$2,471,754,307",1997,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Star Wars: Episode VII - The Force Awakens,/title/tt2488496/?ref_=bo_cso_table_4,"$2,068,454,310",2015,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Avengers: Infinity War,/title/tt4154756/?ref_=bo_cso_table_5,"$2,048,359,754",2018,NaN,NaN,NaN,NaN,NaN,NaN


In [379]:
# move the movie link data from the secondary and tertiary columns into the primary link column
i = 0
for j in range(0, 551):
    if df_new2.isnull().Movie_Page_x[i]:
        df_new2.Movie_Page_x[i] = df_new2.Movie_Page[i]
    if df_new2.isnull().Movie_Page_x[i]:
        df_new2.Movie_Page_x[i] = df_new2.Movie_Page_y[i]
    i +=1
# Then count the NaN cells to make sure there are none
count = df_new2["Movie_Page_x"].isna().sum()
count

In [642]:
from numpy import nan

for y in df_new3.index: 
    if df_new3['Year_Released'][y] == NaN and pd.isna(df_new3['2020_Rank'][y]) == False:
        df_new3.Year_Released.fillna('2020')

NameError: name 'NaN' is not defined

,Lifetime_Rank,Movie_Title,Movie_Page_x,Lifetime_Gross,Year_Released,2019_Rank,Movie_Page_y,2019_Gross,2020_Rank,Movie_Page,2020_Gross
0,1,Avengers: Endgame,/title/tt4154796/?ref_=bo_cso_table_1,"$2,797,800,564",2019,1,/release/rl3059975681/?ref_=bo_yld_table_1,"$858,373,000",NaN,NaN,NaN
1,2,Avatar,/title/tt0499549/?ref_=bo_cso_table_2,"$2,790,439,092",2009,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Titanic,/title/tt0120338/?ref_=bo_cso_table_3,"$2,471,754,307",1997,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Star Wars: Episode VII - The Force Awakens,/title/tt2488496/?ref_=bo_cso_table_4,"$2,068,454,310",2015,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Avengers: Infinity War,/title/tt4154756/?ref_=bo_cso_table_5,"$2,048,359,754",2018,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
546,NaN,Extra Ordinary,/release/rl2562359809/?ref_=bo_yld_table_196,NaN,NaN,NaN,NaN,NaN,196,/release/rl2562359809/?ref_=bo_yld_table_196,"$164,346"
547,NaN,Palm Springs,/release/rl3175187201/?ref_=bo_yld_table_197,NaN,NaN,NaN,NaN,NaN,197,/release/rl3175187201/?ref_=bo_yld_table_197,"$164,000"
548,NaN,A Call to Spy,/release/rl3796140801/?ref_=bo_yld_table_198,NaN,NaN,NaN,NaN,NaN,198,/release/rl3796140801/?ref_=bo_yld_table_198,"$158,864"
549,NaN,Love Story50th Anniversary,/release/rl934707713/?ref_=bo_yld_table_199,NaN,NaN,NaN,NaN,NaN,199,/release/rl934707713/?ref_=bo_yld_table_199,"$153,504"


In [665]:
# Added the year released for those movies scraped from the 2019 and 2020 highest grossing lists
for y in df_new3.index:
    if pd.isna(df_new3['Year_Released'][y]) == True and pd.isna(df_new3['2020_Rank'][y]) == False:
        df_new3.Year_Released.fillna('2020')

for y in df_new3.index:
    if pd.isna(df_new3['Year_Released'][y]) == True and pd.isna(df_new3['2019_Rank'][y]) == False:
        df_new3.Year_Released.fillna('2019')


didn't


In [677]:
# from numpy import nan
j = 0
for y in df_new3.index:
    if pd.isna(df_new3['2019_Rank'][y]) == False:
        df_new3['2019_Rank'][y] = '2019'
        j +=1
        print(j, df_new3['2019_Rank'][y], df_new3['Year_Released'][y])

1 2019 2019
2 2019 2019
3 2019 1994
4 2019 2019
5 2019 2018
6 2019 2019
7 2019 2019
8 2019 2019
9 2019 2019
10 2019 2019
11 2019 2019
12 2019 2019
13 2019 2018
14 2019 2019
15 2019 2019
16 2019 2019
17 2019 2019
18 2019 2018
19 2019 2018
20 2019 2020
21 2019 2020
22 2019 2020
23 2019 2020
24 2019 2020
25 2019 2020
26 2019 2020
27 2019 2020
28 2019 2020
29 2019 2020
30 2019 2020
31 2019 2020
32 2019 2020
33 2019 2020
34 2019 2020
35 2019 2020
36 2019 2020
37 2019 2020
38 2019 2020
39 2019 2020
40 2019 2020
41 2019 2020
42 2019 2020
43 2019 2020
44 2019 2020
45 2019 2020
46 2019 2020
47 2019 2020
48 2019 2020
49 2019 2020
50 2019 2020
51 2019 2020
52 2019 2020
53 2019 2020
54 2019 2020
55 2019 2020
56 2019 2020
57 2019 2020
58 2019 2020
59 2019 2020
60 2019 2020
61 2019 2020
62 2019 2020
63 2019 2020
64 2019 2020
65 2019 2020
66 2019 2020
67 2019 2020
68 2019 2020
69 2019 2020
70 2019 2020
71 2019 2020
72 2019 2020
73 2019 2020
74 2019 2020
75 2019 2020
76 2019 2020
77 2019 2020
78 2019 

,Lifetime_Rank,Movie_Title,Movie_Page_x,Lifetime_Gross,Year_Released,2019_Rank,Movie_Page_y,2019_Gross,2020_Rank,Movie_Page,2020_Gross
201,NaN,John Wick: Chapter 3 - Parabellum,NaN,NaN,NaN,14,/release/rl1476560385/?ref_=bo_yld_table_14,"$171,015,687",NaN,NaN,NaN
202,NaN,How to Train Your Dragon: The Hidden World,NaN,NaN,NaN,15,/release/rl2606269953/?ref_=bo_yld_table_15,"$160,799,505",NaN,NaN,NaN
203,NaN,The Secret Life of Pets 2,NaN,NaN,NaN,16,/release/rl1719436801/?ref_=bo_yld_table_16,"$157,949,395",NaN,NaN,NaN
204,NaN,Pokémon Detective Pikachu,NaN,NaN,NaN,17,/release/rl17139201/?ref_=bo_yld_table_17,"$144,105,346",NaN,NaN,NaN
205,NaN,Once Upon a Time... In Hollywood,/release/rl520455681/?ref_=bo_yld_table_78,NaN,NaN,18,/release/rl520455681/?ref_=bo_yld_table_18,"$141,076,968",78,/release/rl520455681/?ref_=bo_yld_table_78,"$1,425,760"
206,NaN,Shazam!,NaN,NaN,NaN,19,/release/rl1196197377/?ref_=bo_yld_table_19,"$140,371,656",NaN,NaN,NaN
207,NaN,Knives Out,/release/rl3204875777/?ref_=bo_yld_table_12,NaN,NaN,21,/release/rl3204875777/?ref_=bo_yld_table_21,"$115,711,579",12,/release/rl3204875777/?ref_=bo_yld_table_12,"$49,651,655"
208,NaN,Dumbo,NaN,NaN,NaN,22,/release/rl2675869185/?ref_=bo_yld_table_22,"$114,766,307",NaN,NaN,NaN
209,NaN,Maleficent: Mistress of Evil,/release/rl939755009/?ref_=bo_yld_table_116,NaN,NaN,23,/release/rl939755009/?ref_=bo_yld_table_23,"$113,294,737",116,/release/rl939755009/?ref_=bo_yld_table_116,"$634,868"
210,NaN,Glass,NaN,NaN,NaN,24,/release/rl1518241281/?ref_=bo_yld_table_24,"$111,048,468",NaN,NaN,NaN


In [661]:
df_new3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 551 entries, 0 to 550
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Lifetime_Rank   200 non-null    object
 1   Movie_Title     551 non-null    object
 2   Movie_Page_x    551 non-null    object
 3   Lifetime_Gross  200 non-null    object
 4   Year_Released   200 non-null    object
 5   2019_Rank       201 non-null    object
 6   Movie_Page_y    201 non-null    object
 7   2019_Gross      201 non-null    object
 8   2020_Rank       200 non-null    object
 9   Movie_Page      200 non-null    object
 10  2020_Gross      200 non-null    object
 11  Domestic_Gross  551 non-null    object
 12  Budget          551 non-null    object
 13  MPAA            551 non-null    object
 14  Genres          551 non-null    object
dtypes: object(15)
memory usage: 88.9+ KB


In [485]:
df_new2.head()

,Lifetime_Rank,Movie_Title,Movie_Page_x,Lifetime_Gross,Year_Released,2019_Rank,Movie_Page_y,2019_Gross,2020_Rank,Movie_Page,2020_Gross
0,1,Avengers: Endgame,/title/tt4154796/?ref_=bo_cso_table_1,"$2,797,800,564",2019,1,/release/rl3059975681/?ref_=bo_yld_table_1,"$858,373,000",NaN,NaN,NaN
1,2,Avatar,/title/tt0499549/?ref_=bo_cso_table_2,"$2,790,439,092",2009,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Titanic,/title/tt0120338/?ref_=bo_cso_table_3,"$2,471,754,307",1997,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Star Wars: Episode VII - The Force Awakens,/title/tt2488496/?ref_=bo_cso_table_4,"$2,068,454,310",2015,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Avengers: Infinity War,/title/tt4154756/?ref_=bo_cso_table_5,"$2,048,359,754",2018,NaN,NaN,NaN,NaN,NaN,NaN


In [684]:
df_new4 = df_new2
df_new2.tail()

,Lifetime_Rank,Movie_Title,Movie_Page_x,Lifetime_Gross,Year_Released,2019_Rank,Movie_Page_y,2019_Gross,2020_Rank,Movie_Page,2020_Gross,Domestic_Gross,Budget,MPAA,Genres
546,NaN,Extra Ordinary,/release/rl2562359809/?ref_=bo_yld_table_196,NaN,2020,NaN,NaN,NaN,196,/release/rl2562359809/?ref_=bo_yld_table_196,"$164,346","$164,346","$150,000,000",R,"[Comedy, Fantasy, Horror]"
547,NaN,Palm Springs,/release/rl3175187201/?ref_=bo_yld_table_197,NaN,2020,NaN,NaN,NaN,197,/release/rl3175187201/?ref_=bo_yld_table_197,"$164,000","$164,000","$150,000,000",R,"[Comedy, Fantasy, Mystery, Romance]"
548,NaN,A Call to Spy,/release/rl3796140801/?ref_=bo_yld_table_198,NaN,2020,NaN,NaN,NaN,198,/release/rl3796140801/?ref_=bo_yld_table_198,"$158,864","$159,014","$150,000,000",PG-13,"[Biography, Crime, Drama, Thriller, War]"
549,NaN,Love Story50th Anniversary,/release/rl934707713/?ref_=bo_yld_table_199,NaN,2020,NaN,NaN,NaN,199,/release/rl934707713/?ref_=bo_yld_table_199,"$153,504","$153,504","$150,000,000",PG,"[Drama, Romance]"
550,NaN,The Dark and the Wicked,/release/rl4015883009/?ref_=bo_yld_table_200,NaN,2020,NaN,NaN,NaN,200,/release/rl4015883009/?ref_=bo_yld_table_200,"$149,011","$155,757","$150,000,000",PG,[Horror]


In [685]:
j = 0
for y in df_new3.index:
    if pd.isna(df_new3['2019_Rank'][y]) == False :
        df_new3['Year_Released'][y] = '2019'
        j +=1
        print(j, df_new3['2019_Rank'][y], df_new3['Year_Released'][y])

1 2019 2019
2 2019 2019
3 2019 2019
4 2019 2019
5 2019 2019
6 2019 2019
7 2019 2019
8 2019 2019
9 2019 2019
10 2019 2019
11 2019 2019
12 2019 2019
13 2019 2019
14 2019 2019
15 2019 2019
16 2019 2019
17 2019 2019
18 2019 2019
19 2019 2019
20 2019 2019
21 2019 2019
22 2019 2019
23 2019 2019
24 2019 2019
25 2019 2019
26 2019 2019
27 2019 2019
28 2019 2019
29 2019 2019
30 2019 2019
31 2019 2019
32 2019 2019
33 2019 2019
34 2019 2019
35 2019 2019
36 2019 2019
37 2019 2019
38 2019 2019
39 2019 2019
40 2019 2019
41 2019 2019
42 2019 2019
43 2019 2019
44 2019 2019
45 2019 2019
46 2019 2019
47 2019 2019
48 2019 2019
49 2019 2019
50 2019 2019
51 2019 2019
52 2019 2019
53 2019 2019
54 2019 2019
55 2019 2019
56 2019 2019
57 2019 2019
58 2019 2019
59 2019 2019
60 2019 2019
61 2019 2019
62 2019 2019
63 2019 2019
64 2019 2019
65 2019 2019
66 2019 2019
67 2019 2019
68 2019 2019
69 2019 2019
70 2019 2019
71 2019 2019
72 2019 2019
73 2019 2019
74 2019 2019
75 2019 2019
76 2019 2019
77 2019 2019
78 2019 

In [686]:
j = 0
for y in df_new3.index:
    if pd.isna(df_new3['2020_Rank'][y]) == False :
#         df_new3['Year_Released'][y] = '2020'
        j +=1
        print(j, df_new3['2020_Rank'][y], df_new3['Year_Released'][y])

1 13 2019
2 76 2019
3 5 2019
4 105 2017
5 4 2019
6 78 2019
7 12 2019
8 116 2019
9 39 2019
10 129 2019
11 187 2019
12 52 2019
13 120 2019
14 161 2019
15 127 2019
16 62 2019
17 8 2019
18 15 2019
19 101 2019
20 21 2019
21 163 2019
22 19 2019
23 37 2019
24 48 2019
25 34 2019
26 54 2019
27 192 2019
28 134 2019
29 121 2019
30 111 2019
31 94 2019
32 1 2020
33 2 2020
34 3 2020
35 6 2020
36 7 2020
37 9 2020
38 10 2020
39 11 2020
40 14 2020
41 16 2020
42 17 2020
43 18 2020
44 20 2020
45 22 2020
46 23 2020
47 24 2020
48 25 2020
49 26 2020
50 27 2020
51 28 2020
52 29 2020
53 30 2020
54 31 2020
55 32 2020
56 33 2020
57 35 2020
58 36 2020
59 38 2020
60 40 2020
61 41 2020
62 42 2020
63 43 2020
64 44 2020
65 45 2020
66 46 2020
67 47 2020
68 49 2020
69 50 2020
70 51 2020
71 53 2020
72 55 2020
73 56 2020
74 57 2020
75 58 2020
76 59 2020
77 60 2020
78 61 2020
79 63 2020
80 64 2020
81 65 2020
82 66 2020
83 67 2020
84 68 2020
85 69 2020
86 70 2020
87 71 2020
88 72 2020
89 73 2020
90 74 2020
91 75 2020
92 7

In [707]:
# df_new4.iloc[200]
# df_new4.info()
df_new4[df_new4['Lifetime_Gross'][0]] = df_new4[df_new4['Lifetime_Gross'][0]].replace('[\$,]', '', regex=True).astype(float)

KeyError: '$2,797,800,564'

In [709]:
#converted strings in Lifetime_Gross column to integers, removing the commas and dollar signs
i = 0
for y in df_new4:
    df_new4['Lifetime_Gross'][i] = df_new4['Lifetime_Gross'][i].replace(',', '')
    df_new4['Lifetime_Gross'][i] = df_new4['Lifetime_Gross'][i].replace('$', '')
    i +=1
df_new4.head()

,Lifetime_Rank,Movie_Title,Movie_Page_x,Lifetime_Gross,Year_Released,2019_Rank,Movie_Page_y,2019_Gross,2020_Rank,Movie_Page,2020_Gross,Domestic_Gross,Budget,MPAA,Genres
0,1,Avengers: Endgame,/title/tt4154796/?ref_=bo_cso_table_1,2797800564,2019,2019,/release/rl3059975681/?ref_=bo_yld_table_1,"$858,373,000",NaN,NaN,NaN,"$858,373,000","$356,000,000",PG-13,"[Action, Adventure, Drama, Sci-Fi]"
1,2,Avatar,/title/tt0499549/?ref_=bo_cso_table_2,2790439092,2009,NaN,NaN,NaN,NaN,NaN,NaN,"$760,507,625","$237,000,000",PG-13,"[Action, Adventure, Fantasy, Sci-Fi]"
2,3,Titanic,/title/tt0120338/?ref_=bo_cso_table_3,2471754307,1997,NaN,NaN,NaN,NaN,NaN,NaN,"$659,363,944","$200,000,000",PG-13,"[Drama, Romance]"
3,4,Star Wars: Episode VII - The Force Awakens,/title/tt2488496/?ref_=bo_cso_table_4,2068454310,2015,NaN,NaN,NaN,NaN,NaN,NaN,"$936,662,225","$245,000,000",PG-13,"[Action, Adventure, Sci-Fi]"
4,5,Avengers: Infinity War,/title/tt4154756/?ref_=bo_cso_table_5,2048359754,2018,NaN,NaN,NaN,NaN,NaN,NaN,"$678,815,482","$245,000,000",PG-13,"[Action, Adventure, Sci-Fi]"


In [ ]:
#converted strings in Domestic Gross and Budget columns to integers, removing the commas and dollar signs
i = 0
for y in df_new4:
    df_new4['Domestic_Gross'][i] = df_new4['Domestic_Gross'][i].replace(',', '')
    df_new4['Domestic_Gross'][i] = df_new4['Domestic_Gross'][i].replace('$', '')
    i +=1
    
i = 0
for y in df_new4:
    df_new4['Budget'][i] = df_new4['Budget'][i].replace(',', '')
    df_new4['Budget'][i] = df_new4['Lifetime_Gross'][i].replace('$', '')
    i +=1

df_new4.head()